In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

# Data Preprocessing for Matrix User x Item

In [2]:
#Source Code for data Processing
#https://gist.github.com/victorkohler/f48ea6512058719ba52053851fedc745

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [3]:
#np.shape(train_data)
np.shape(test_data)

(3782335, 12)

In [4]:
#train = train_data.append(test_data)
#np.shape(train)

In [5]:
item_metadata = pd.read_csv('data/item_metadata.csv', sep=',', engine='python')

In [6]:
train_v2 = train_data[train_data['action_type']=='clickout item']
train_v2= train_v2[train_v2.reference.notnull()]
train_v2.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
29,01R92KASN25O,62991f7c78f27,1541055672,8,clickout item,7818446,DE,"Luebeck, Germany",mobile,NaN,7818446|51315|2133708|119638|86359|133581|6688...,163|75|93|98|169|201|129|117|108|170|133|80|87...
32,026J2T90PT57,67c4d45f56146,1541105688,1,clickout item,2681512,MX,"Ezequiel Montes, Mexico",mobile,NaN,6010044|5433188|5156000|4780400|5137162|992819...,38|35|54|37|73|37|37|30|71|75|64|26|25|37|40|3...
39,032W52V15J3A,e84393cf62d13,1541102603,7,clickout item,929533,HR,"Cancun, Mexico",mobile,NaN,3132957|100226|1954167|9462680|2776177|929533|...,25|324|67|52|57|21|90|240|64|37|70|60|167|45|6...
73,03F93632Y45U,f09aa370de0af,1541109160,34,clickout item,8496296,US,"San Diego, USA",mobile,NaN,9112592|102414|9140448|5654906|4341120|9739716...,263|114|263|273|122|194|236|161|219|113|110|19...
393,08527D0MD8VC,3e2a95e5727f9,1541099104,320,clickout item,8266350,ES,"Lisbon, Portugal",desktop,NaN,3505518|1834297|4837106|3990396|2042495|486747...,47|50|45|40|45|40|40|50|50|45|50|45|38|49|48|3...


In [7]:
train_v2.dtypes

user_id            object
session_id         object
timestamp           int64
step                int64
action_type        object
reference          object
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtype: object

In [8]:
train_v2["reference"]= train_v2["reference"].astype(int)
train_v2["step"]= train_v2["step"].astype(int)
train_v2.dtypes

user_id            object
session_id         object
timestamp           int64
step                int64
action_type        object
reference           int64
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtype: object

In [9]:
train_v2.head()
#np.shape(train_v2)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
29,01R92KASN25O,62991f7c78f27,1541055672,8,clickout item,7818446,DE,"Luebeck, Germany",mobile,NaN,7818446|51315|2133708|119638|86359|133581|6688...,163|75|93|98|169|201|129|117|108|170|133|80|87...
32,026J2T90PT57,67c4d45f56146,1541105688,1,clickout item,2681512,MX,"Ezequiel Montes, Mexico",mobile,NaN,6010044|5433188|5156000|4780400|5137162|992819...,38|35|54|37|73|37|37|30|71|75|64|26|25|37|40|3...
39,032W52V15J3A,e84393cf62d13,1541102603,7,clickout item,929533,HR,"Cancun, Mexico",mobile,NaN,3132957|100226|1954167|9462680|2776177|929533|...,25|324|67|52|57|21|90|240|64|37|70|60|167|45|6...
73,03F93632Y45U,f09aa370de0af,1541109160,34,clickout item,8496296,US,"San Diego, USA",mobile,NaN,9112592|102414|9140448|5654906|4341120|9739716...,263|114|263|273|122|194|236|161|219|113|110|19...
393,08527D0MD8VC,3e2a95e5727f9,1541099104,320,clickout item,8266350,ES,"Lisbon, Portugal",desktop,NaN,3505518|1834297|4837106|3990396|2042495|486747...,47|50|45|40|45|40|40|50|50|45|50|45|38|49|48|3...


In [10]:
#train_v3 = train_v2.head(10000)
train_v3 = train_v2
train_v3.head()
np.shape(train_v3)

(1586586, 12)

In [11]:
data = train_v3[['user_id','reference']]
data.head()
#np.shape(data)

,user_id,reference
29,01R92KASN25O,7818446
32,026J2T90PT57,2681512
39,032W52V15J3A,929533
73,03F93632Y45U,8496296
393,08527D0MD8VC,8266350


In [12]:
data=data.groupby(['user_id','reference']).size().reset_index()
data.columns = ['user', 'item', 'click_count']
data.head()

,user,item,click_count
0,0001VQMGUI65,477811,1
1,0001VQMGUI65,950829,1
2,0001VQMGUI65,2019467,1
3,0001VQMGUI65,3133074,2
4,0003QTCX5MJX,2195060,1


In [13]:
user_id_table = data['user']
user_id_table = pd.DataFrame(user_id_table)
user_id_table = pd.DataFrame(user_id_table['user'].unique())
user_id_table.columns = ['user']
user_id_table['user_id'] = dict(enumerate(user_id_table['user'], start=0))
user_id_table.head()
#np.shape(user_id_table)

,user,user_id
0,0001VQMGUI65,0
1,0003QTCX5MJX,1
2,0004IOZI7CKF,2
3,0004WCFRV3FB,3
4,0006W0R5A5V8,4


In [14]:
item_id_table = data['item']
item_id_table = pd.DataFrame(item_id_table)
item_id_table = pd.DataFrame(item_id_table['item'].unique())
item_id_table.columns = ['item']
item_id_table['item_id'] = dict(enumerate(item_id_table['item'], start=0))
item_id_table.head()
#np.shape(item_id_table)

,item,item_id
0,477811,0
1,950829,1
2,2019467,2
3,3133074,3
4,2195060,4


In [15]:
data = data.merge(user_id_table, how='left', left_on='user', right_on = 'user').merge(item_id_table, how='left', left_on='item', right_on = 'item')
data.head()
#data = data[["user_id", "item_id","clickcount"]]
#data.head()

,user,item,click_count,user_id,item_id
0,0001VQMGUI65,477811,1,0,0
1,0001VQMGUI65,950829,1,0,1
2,0001VQMGUI65,2019467,1,0,2
3,0001VQMGUI65,3133074,2,0,3
4,0003QTCX5MJX,2195060,1,1,4


In [16]:
data2 = data[["user_id", "item_id","click_count"]]
data2.head()

,user_id,item_id,click_count
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,2
4,1,4,1


In [17]:
#Based on the implementation from https://github.com/ivallesp/deep_matrix_factorization
def batching(list_of_iterables, n=1, infinite=False, return_incomplete_batches=False):
    list_of_iterables = [list_of_iterables] if type(list_of_iterables) is not list else list_of_iterables
    assert(len({len(it) for it in list_of_iterables}) == 1)
    l = len(list_of_iterables[0])
    while 1:
        for ndx in range(0, l, n):
            if not return_incomplete_batches:
                if (ndx+n) > l:
                    break
            yield [iterable[ndx:min(ndx + n, l)] for iterable in list_of_iterables]

        if not infinite:
            break

In [18]:
#Based on the implementation from https://github.com/ivallesp/deep_matrix_factorization
import tensorflow as tf
import numpy as np

class DeepFactorization():
    def __init__(self, n_users, n_items, emb_size, lr=0.001, _lambda = 0.1):
        self.lr = lr
        self._lambda = _lambda
        tf.reset_default_graph()
        self.ph_keep_prob = tf.placeholder(dtype=tf.float32, shape=(None), name="keep_prob")
        self.ph_u_ids=tf.placeholder(dtype=tf.int32, shape=(None,), name="u_ids_ph")
        self.ph_i_ids=tf.placeholder(dtype=tf.int32, shape=(None,), name="u_ids_ph")
        self.ph_y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="y")
        self.u_emb = tf.get_variable(shape=(n_users, emb_size), name="U_Embedding")
        self.i_emb = tf.get_variable(shape=(n_items, emb_size), name="I_Embedding")
        self.u_bias = tf.get_variable(shape=(n_users, 1), name="U_Bias")
        self.i_bias = tf.get_variable(shape=(n_items, 1), name="I_Bias")
        u=tf.nn.embedding_lookup(self.u_emb, self.ph_u_ids) #+ tf.nn.embedding_lookup(self.u_bias, self.ph_u_ids) 
        i=tf.nn.embedding_lookup(self.i_emb, self.ph_i_ids) #+ tf.nn.embedding_lookup(self.i_bias, self.ph_i_ids)
        ub=tf.nn.embedding_lookup(self.u_bias, self.ph_u_ids)
        ib=tf.nn.embedding_lookup(self.i_bias, self.ph_i_ids)
        ub=tf.nn.dropout(ub, self.ph_keep_prob)
        ib=tf.nn.dropout(ib, self.ph_keep_prob)
        u=tf.nn.dropout(u, self.ph_keep_prob)
        i=tf.nn.dropout(i, self.ph_keep_prob)
        self.mfac = tf.reduce_sum(u*i + ub + ib, axis=1, keepdims=True)
        self.mfac = tf.nn.dropout(self.mfac, self.ph_keep_prob)
        emb = tf.concat([u, i], axis=1)
        emb = tf.nn.dropout(emb, self.ph_keep_prob)
        h = tf.layers.Dense(128, tf.nn.swish)(emb)
        h = tf.nn.dropout(h, self.ph_keep_prob)
        h = tf.layers.Dense(64, tf.nn.swish)(h)
        h = tf.nn.dropout(h, self.ph_keep_prob)
        h = tf.layers.Dense(32, tf.nn.swish)(h)
        h = tf.nn.dropout(h, self.ph_keep_prob)
        self.output = tf.layers.Dense(1)(h) + self.mfac
        self.loss = tf.losses.mean_squared_error(self.ph_y, self.output)
        self.op = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(self.loss)
        self.sess=tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def fit(self, x, y, batch_size):
        batcher = batching([list(x), list(y)], n=batch_size, return_incomplete_batches=True)
        for batch_x, batch_y in batcher:
            self.train_on_batch(batch_x, batch_y)
        
    def train_on_batch(self, x, y):
        u_ids, i_ids = list(zip(*list(x)))                              
        self.sess.run(self.op, feed_dict={self.ph_u_ids: u_ids, self.ph_i_ids: i_ids, self.ph_y: y,
                                          self.ph_keep_prob: 1-self._lambda})
        
    
    def predict(self, x, batch_size):
        batcher = batching([list(x)], n=batch_size, return_incomplete_batches=True)
        preds = []
        for batch_x in batcher:
            batch_x = batch_x[0]
            u_ids, i_ids = list(zip(*list(batch_x)))
            preds.append(self.sess.run(self.output, feed_dict={self.ph_u_ids: u_ids, self.ph_i_ids: i_ids,
                                                      self.ph_keep_prob: 1.0}))
        preds = np.row_stack(preds)
        return preds
    
    def evaluate(self, x, y, batch_size):
        return np.mean((np.round(self.predict(x, batch_size)) - y)**2)

In [19]:
data.head()

,user,item,click_count,user_id,item_id
0,0001VQMGUI65,477811,1,0,0
1,0001VQMGUI65,950829,1,0,1
2,0001VQMGUI65,2019467,1,0,2
3,0001VQMGUI65,3133074,2,0,3
4,0003QTCX5MJX,2195060,1,1,4


In [20]:
# Load the data and calculate the users set and items set cardinalities
import itertools

original_itemid = data["item"].tolist()

df = data2
df = df.iloc[:5000,:5000] #slicing for testing smaller sample
U_cardinality,I_cardinality = np.unique(df['user_id']).shape[0], np.unique(df['item_id']).shape[0]
mat = df.values
train_mat = mat
x_train=(train_mat[:,:2])
y_train=(train_mat[:,2:])

DF = DeepFactorization(n_users=U_cardinality, n_items=I_cardinality, emb_size=3, lr=0.002, _lambda=0.1)
losses_train = [DF.evaluate(x_train, y_train, batch_size=200)] # Add the initial loss (w. random weights)
for i in range(50): # Run for 50 epochs
    DF.fit(x_train, y_train, batch_size=128)
    #losses_train.append(DF.evaluate(x_train, y_train, batch_size=128)) # Compute the train performance

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [21]:
def trivago_item_to_model_item(trivago_item_id):
    """
    Converts an item id on the trivago format over to the id in the model.
    
    trivago_item_id: int
    returns: int
    """
    for model_item_id in range(len(original_itemid)):
        if original_itemid[model_item_id] == trivago_item_id:
            return model_item_id
    raise Exception("There's no item with trivago item id", trivago_item_id)

In [22]:
def model_item_to_trivago_item(model_item_id):
    """
    Converts an item id on the model format over to the id in trivago.
    
    model_item_id: int
    returns: String
    """
    
    try:
        return original_itemid[model_item_id]
    except:
        raise Exception("There's no item with model item id", model_item_id)

In [23]:
def trivago_user_to_model_user(trivago_user_id):
    """
    Converts a user id on the trivago format over to the id in the model.
    
    trivago_user_id: int
    returns: int
    """
    original_userid = user_id_table["user"].tolist()
    for model_user_id in range(len(original_userid)):
        if original_userid[model_user_id] == trivago_user_id:
            return model_user_id
    raise Exception("There's no user with trivago user id", trivago_user_id)

In [24]:
def model_user_to_trivago_user(model_user_id):
    """
    Converts a user id on the model format over to the id in trivago.
    
    model_item_id: int
    returns: String
    """
    original_userid = user_id_table["user"].tolist()
    try:
        return original_userid[model_user_id]
    except:
        raise Exception("There's no model user with user id", model_user_id)

In [25]:
trivago_item_to_model_item(10875780)
model_item_to_trivago_item(167)
trivago_user_to_model_user("0004IOZI7CKF")
model_user_to_trivago_user(7)

'0008W02TLRNR'

In [33]:
import random
import csv
import operator


class Submission:
    """
    Find the predicted top ratings and writes it to a file in the format specified by the RecSys challenge.
    """

    def sort_items(self, user, items):
        """
        Given a user and a list of items, this method predicts the user's rating for these items and 
        return a sorted list of the items in descending order of the item's predicted ratings.
        """
        
        try:
            model_user_id = trivago_user_to_model_user(user)
        except:
            # If the user is not known to the system, we submit the accomodations in the impression list
            return items
        
        x = (model_user_id,)
        y = list(range(50))
        
        # DF.predict crashes with this user InvalidArgumentError: indices[0] = 2797 is not in [0, 2789)
        if model_user_id > 2789:
            return items

        user_item_iterable = list(itertools.product(x, y))
        print(user_item_iterable)
        predictions = DF.predict(user_item_iterable,200)
        
        # Creates a dict with item id as key and predicted rating as value
        item_dict = dict()
        for j in range(len(predictions)):
            item_dict[original_itemid[j]] = predictions[j][0]
        
        # Creates a list of the top 25 recommended hotels
        sorted_item_list = sorted(item_dict, key=item_dict.get, reverse=True)[:25]
        
        # Converts items in list from int to String
        for i in range(len(sorted_item_list)):
            sorted_item_list[i] = str(sorted_item_list[i])
        
        return sorted_item_list
    
    def write_header_line(self):
        with open("dmf/submission.txt", mode="w") as submission:
            recommendation_writer = csv.writer(submission, delimiter=',', quotechar='"', 
                                               quoting=csv.QUOTE_MINIMAL)
            recommendation_writer.writerow(["user_id", "session_id", "timestamp", "step", 
                                            "item_recommendations"])

    def write_line(self, user_id, session_id, timestamp, step, recommendations):
        """
        Writes a line in the submission file.
        :param recommendations: The list of recommendations in descending order
        """
        
        with open("dmf/submission.txt", mode="a") as submission:
            recommendation_writer = csv.writer(submission, delimiter=',', quotechar='"', 
                                               quoting=csv.QUOTE_MINIMAL)
            recommendation_writer.writerow([user_id, session_id, timestamp, step, " ".join(recommendations)])

    def write_submission_file(self):
        """
        Generate and writes the file that is submitted to the RecSys challenge.
        """
        self.write_header_line()

        with open("data/test.csv") as test:
            reader = csv.reader(test)
            for row in reader:
                action_type = row[4]
                if action_type == "clickout item":
                    reference = row[5]
                    if not reference:
                        # This is the line is missing the click-out item. Generating recommendations for 
                        # this user.
                        user_id, session_id, timestamp, step = row[0:4]
                        items = row[10].split("|")
                        recommendations = self.sort_items(user_id, items)

                        # Writes the result to file
                        self.write_line(user_id, session_id, timestamp, step, recommendations)



In [ ]:
sub = Submission()
sub.write_submission_file()

[(6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (6, 16), (6, 17), (6, 18), (6, 19), (6, 20), (6, 21), (6, 22), (6, 23), (6, 24), (6, 25), (6, 26), (6, 27), (6, 28), (6, 29), (6, 30), (6, 31), (6, 32), (6, 33), (6, 34), (6, 35), (6, 36), (6, 37), (6, 38), (6, 39), (6, 40), (6, 41), (6, 42), (6, 43), (6, 44), (6, 45), (6, 46), (6, 47), (6, 48), (6, 49)]
[(23, 0), (23, 1), (23, 2), (23, 3), (23, 4), (23, 5), (23, 6), (23, 7), (23, 8), (23, 9), (23, 10), (23, 11), (23, 12), (23, 13), (23, 14), (23, 15), (23, 16), (23, 17), (23, 18), (23, 19), (23, 20), (23, 21), (23, 22), (23, 23), (23, 24), (23, 25), (23, 26), (23, 27), (23, 28), (23, 29), (23, 30), (23, 31), (23, 32), (23, 33), (23, 34), (23, 35), (23, 36), (23, 37), (23, 38), (23, 39), (23, 40), (23, 41), (23, 42), (23, 43), (23, 44), (23, 45), (23, 46), (23, 47), (23, 48), (23, 49)]
[(28, 0), (28, 1), (28, 2), (28, 3), (28, 4), (28, 5), (28, 6), (28,

[(421, 0), (421, 1), (421, 2), (421, 3), (421, 4), (421, 5), (421, 6), (421, 7), (421, 8), (421, 9), (421, 10), (421, 11), (421, 12), (421, 13), (421, 14), (421, 15), (421, 16), (421, 17), (421, 18), (421, 19), (421, 20), (421, 21), (421, 22), (421, 23), (421, 24), (421, 25), (421, 26), (421, 27), (421, 28), (421, 29), (421, 30), (421, 31), (421, 32), (421, 33), (421, 34), (421, 35), (421, 36), (421, 37), (421, 38), (421, 39), (421, 40), (421, 41), (421, 42), (421, 43), (421, 44), (421, 45), (421, 46), (421, 47), (421, 48), (421, 49)]
[(475, 0), (475, 1), (475, 2), (475, 3), (475, 4), (475, 5), (475, 6), (475, 7), (475, 8), (475, 9), (475, 10), (475, 11), (475, 12), (475, 13), (475, 14), (475, 15), (475, 16), (475, 17), (475, 18), (475, 19), (475, 20), (475, 21), (475, 22), (475, 23), (475, 24), (475, 25), (475, 26), (475, 27), (475, 28), (475, 29), (475, 30), (475, 31), (475, 32), (475, 33), (475, 34), (475, 35), (475, 36), (475, 37), (475, 38), (475, 39), (475, 40), (475, 41), (475, 

[(877, 0), (877, 1), (877, 2), (877, 3), (877, 4), (877, 5), (877, 6), (877, 7), (877, 8), (877, 9), (877, 10), (877, 11), (877, 12), (877, 13), (877, 14), (877, 15), (877, 16), (877, 17), (877, 18), (877, 19), (877, 20), (877, 21), (877, 22), (877, 23), (877, 24), (877, 25), (877, 26), (877, 27), (877, 28), (877, 29), (877, 30), (877, 31), (877, 32), (877, 33), (877, 34), (877, 35), (877, 36), (877, 37), (877, 38), (877, 39), (877, 40), (877, 41), (877, 42), (877, 43), (877, 44), (877, 45), (877, 46), (877, 47), (877, 48), (877, 49)]
[(884, 0), (884, 1), (884, 2), (884, 3), (884, 4), (884, 5), (884, 6), (884, 7), (884, 8), (884, 9), (884, 10), (884, 11), (884, 12), (884, 13), (884, 14), (884, 15), (884, 16), (884, 17), (884, 18), (884, 19), (884, 20), (884, 21), (884, 22), (884, 23), (884, 24), (884, 25), (884, 26), (884, 27), (884, 28), (884, 29), (884, 30), (884, 31), (884, 32), (884, 33), (884, 34), (884, 35), (884, 36), (884, 37), (884, 38), (884, 39), (884, 40), (884, 41), (884, 

[(1263, 0), (1263, 1), (1263, 2), (1263, 3), (1263, 4), (1263, 5), (1263, 6), (1263, 7), (1263, 8), (1263, 9), (1263, 10), (1263, 11), (1263, 12), (1263, 13), (1263, 14), (1263, 15), (1263, 16), (1263, 17), (1263, 18), (1263, 19), (1263, 20), (1263, 21), (1263, 22), (1263, 23), (1263, 24), (1263, 25), (1263, 26), (1263, 27), (1263, 28), (1263, 29), (1263, 30), (1263, 31), (1263, 32), (1263, 33), (1263, 34), (1263, 35), (1263, 36), (1263, 37), (1263, 38), (1263, 39), (1263, 40), (1263, 41), (1263, 42), (1263, 43), (1263, 44), (1263, 45), (1263, 46), (1263, 47), (1263, 48), (1263, 49)]
[(1315, 0), (1315, 1), (1315, 2), (1315, 3), (1315, 4), (1315, 5), (1315, 6), (1315, 7), (1315, 8), (1315, 9), (1315, 10), (1315, 11), (1315, 12), (1315, 13), (1315, 14), (1315, 15), (1315, 16), (1315, 17), (1315, 18), (1315, 19), (1315, 20), (1315, 21), (1315, 22), (1315, 23), (1315, 24), (1315, 25), (1315, 26), (1315, 27), (1315, 28), (1315, 29), (1315, 30), (1315, 31), (1315, 32), (1315, 33), (1315, 34)

[(1630, 0), (1630, 1), (1630, 2), (1630, 3), (1630, 4), (1630, 5), (1630, 6), (1630, 7), (1630, 8), (1630, 9), (1630, 10), (1630, 11), (1630, 12), (1630, 13), (1630, 14), (1630, 15), (1630, 16), (1630, 17), (1630, 18), (1630, 19), (1630, 20), (1630, 21), (1630, 22), (1630, 23), (1630, 24), (1630, 25), (1630, 26), (1630, 27), (1630, 28), (1630, 29), (1630, 30), (1630, 31), (1630, 32), (1630, 33), (1630, 34), (1630, 35), (1630, 36), (1630, 37), (1630, 38), (1630, 39), (1630, 40), (1630, 41), (1630, 42), (1630, 43), (1630, 44), (1630, 45), (1630, 46), (1630, 47), (1630, 48), (1630, 49)]
[(1650, 0), (1650, 1), (1650, 2), (1650, 3), (1650, 4), (1650, 5), (1650, 6), (1650, 7), (1650, 8), (1650, 9), (1650, 10), (1650, 11), (1650, 12), (1650, 13), (1650, 14), (1650, 15), (1650, 16), (1650, 17), (1650, 18), (1650, 19), (1650, 20), (1650, 21), (1650, 22), (1650, 23), (1650, 24), (1650, 25), (1650, 26), (1650, 27), (1650, 28), (1650, 29), (1650, 30), (1650, 31), (1650, 32), (1650, 33), (1650, 34)

[(1847, 0), (1847, 1), (1847, 2), (1847, 3), (1847, 4), (1847, 5), (1847, 6), (1847, 7), (1847, 8), (1847, 9), (1847, 10), (1847, 11), (1847, 12), (1847, 13), (1847, 14), (1847, 15), (1847, 16), (1847, 17), (1847, 18), (1847, 19), (1847, 20), (1847, 21), (1847, 22), (1847, 23), (1847, 24), (1847, 25), (1847, 26), (1847, 27), (1847, 28), (1847, 29), (1847, 30), (1847, 31), (1847, 32), (1847, 33), (1847, 34), (1847, 35), (1847, 36), (1847, 37), (1847, 38), (1847, 39), (1847, 40), (1847, 41), (1847, 42), (1847, 43), (1847, 44), (1847, 45), (1847, 46), (1847, 47), (1847, 48), (1847, 49)]
[(1859, 0), (1859, 1), (1859, 2), (1859, 3), (1859, 4), (1859, 5), (1859, 6), (1859, 7), (1859, 8), (1859, 9), (1859, 10), (1859, 11), (1859, 12), (1859, 13), (1859, 14), (1859, 15), (1859, 16), (1859, 17), (1859, 18), (1859, 19), (1859, 20), (1859, 21), (1859, 22), (1859, 23), (1859, 24), (1859, 25), (1859, 26), (1859, 27), (1859, 28), (1859, 29), (1859, 30), (1859, 31), (1859, 32), (1859, 33), (1859, 34)

[(2157, 0), (2157, 1), (2157, 2), (2157, 3), (2157, 4), (2157, 5), (2157, 6), (2157, 7), (2157, 8), (2157, 9), (2157, 10), (2157, 11), (2157, 12), (2157, 13), (2157, 14), (2157, 15), (2157, 16), (2157, 17), (2157, 18), (2157, 19), (2157, 20), (2157, 21), (2157, 22), (2157, 23), (2157, 24), (2157, 25), (2157, 26), (2157, 27), (2157, 28), (2157, 29), (2157, 30), (2157, 31), (2157, 32), (2157, 33), (2157, 34), (2157, 35), (2157, 36), (2157, 37), (2157, 38), (2157, 39), (2157, 40), (2157, 41), (2157, 42), (2157, 43), (2157, 44), (2157, 45), (2157, 46), (2157, 47), (2157, 48), (2157, 49)]
[(2187, 0), (2187, 1), (2187, 2), (2187, 3), (2187, 4), (2187, 5), (2187, 6), (2187, 7), (2187, 8), (2187, 9), (2187, 10), (2187, 11), (2187, 12), (2187, 13), (2187, 14), (2187, 15), (2187, 16), (2187, 17), (2187, 18), (2187, 19), (2187, 20), (2187, 21), (2187, 22), (2187, 23), (2187, 24), (2187, 25), (2187, 26), (2187, 27), (2187, 28), (2187, 29), (2187, 30), (2187, 31), (2187, 32), (2187, 33), (2187, 34)

[(2729, 0), (2729, 1), (2729, 2), (2729, 3), (2729, 4), (2729, 5), (2729, 6), (2729, 7), (2729, 8), (2729, 9), (2729, 10), (2729, 11), (2729, 12), (2729, 13), (2729, 14), (2729, 15), (2729, 16), (2729, 17), (2729, 18), (2729, 19), (2729, 20), (2729, 21), (2729, 22), (2729, 23), (2729, 24), (2729, 25), (2729, 26), (2729, 27), (2729, 28), (2729, 29), (2729, 30), (2729, 31), (2729, 32), (2729, 33), (2729, 34), (2729, 35), (2729, 36), (2729, 37), (2729, 38), (2729, 39), (2729, 40), (2729, 41), (2729, 42), (2729, 43), (2729, 44), (2729, 45), (2729, 46), (2729, 47), (2729, 48), (2729, 49)]
[(2751, 0), (2751, 1), (2751, 2), (2751, 3), (2751, 4), (2751, 5), (2751, 6), (2751, 7), (2751, 8), (2751, 9), (2751, 10), (2751, 11), (2751, 12), (2751, 13), (2751, 14), (2751, 15), (2751, 16), (2751, 17), (2751, 18), (2751, 19), (2751, 20), (2751, 21), (2751, 22), (2751, 23), (2751, 24), (2751, 25), (2751, 26), (2751, 27), (2751, 28), (2751, 29), (2751, 30), (2751, 31), (2751, 32), (2751, 33), (2751, 34)